## Project Summary

Top-level stats, motivations, and results that are otherwise difficult to compile from individual project files.

Outline:

- Corpora selection and import
    - Sources:
        - Entrez corpus
        - PMC OA corpus
    - Discuss filtering to relevant documents
- Ontology Integration
    - Discuss CL, CKR, and PRO imports
        - Explain choices for root nodes and conflict resolution
    - Show how synonym lists are prioritized across sources
- Corpus Processing
    - Outline tokenization and entity tagging process
    - Show ScispaCy JNLPBA tagger examples
- Cell Phenotype Inference
    - Frame as tangent to cell type ontology expansion
    - Discuss modalities in cell type references
    - Explain protein tokenization with examples
    - Show results from analyzing subtokens in JNLPBA tags
- Candidate Generation
    - Explain how candidates are paired within sentence and stored in snorkel db
- Candidate Annotation
    - Give stats on initial BRAT annotations and relation classes
    - Provide info on test set annotations (outside of BRAT)
- Labeling Functions
    - Reference regex rules built during annotation
    - Reference other simple heuristics
    - Show some statistics on dev set LFs 
    - Discuss avoidance of advanced heuristics in favor of dev set modeling
        - Show results from RNNs over tokenization, entity marking, embedding, and position feature strategies
        - Show example feature inputs for different settings
        - Explain vocab clipping for pre-trained embeddings
        - Reference modeling code
- Discriminative Model Training
    - Explain where final model is trained and how it is used for each task
- ImmuneXpresso Comparison
    - Show final results as compared to scrape from iX